In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import pandas as pd
from ortools.sat.python import cp_model


family_data_path = '../input/santa-workshop-tour-2019/family_data.csv'
submission_path = '../input/santa-workshop-tour-2019/sample_submission.csv'
pd.set_option('display.max_rows', 500)

In [ ]:
N_FAMILIES = 5000
N_DAYS = 100
N_SLOTS = 9
SCALER = 1
all_families = range(N_FAMILIES)
all_days = range(N_DAYS)

In [ ]:
def input_data():
    input_family_data = pd.read_csv(family_data_path, index_col='family_id')
    family_size = input_family_data.n_people.values.astype(np.int8)
    family_choices = np.array(input_family_data[['choice_{}'.format(i) for i in range(10)]])
    family_slots_matrix = np.zeros((5000, 100))
    for fam in range(5000):
        for choice in range(5):
            family_slots_matrix[fam, family_choices[fam, choice]-1] = choice+1
    return family_size, family_choices, family_slots_matrix

In [ ]:
def get_penalty(n, choice):
    penalty = None
    if choice == 0:
        penalty = 0
    elif choice == 1:
        penalty = 50
    elif choice == 2:
        penalty = 50 + 9 * n
    elif choice == 3:
        penalty = 100 + 9 * n
    elif choice == 4:
        penalty = 200 + 9 * n
    elif choice == 5:
        penalty = 200 + 18 * n
    elif choice == 6:
        penalty = 300 + 18 * n
    elif choice == 7:
        penalty = 300 + 36 * n
    elif choice == 8:
        penalty = 400 + 36 * n
    elif choice == 9:
        penalty = 500 + 36 * n + 199 * n
    else:
        penalty = 500 + 36 * n + 398 * n
    return penalty

def get_preference_cost_matrix(data, family_size):
    cost_matrix = np.zeros((N_FAMILIES, N_DAYS), dtype=np.int64)
    for i in range(N_FAMILIES):
        desired = data.values[i, :-1]
        cost_matrix[i, :] = get_penalty(family_size[i], 10)
        for j, day in enumerate(desired):
            cost_matrix[i, day-1] = get_penalty(family_size[i], j)
    return cost_matrix

def get_accounting_cost_matrix():
    ac = np.zeros((1500, 1500), dtype=np.float64)
    for n in range(ac.shape[0]):
        for n_p1 in range(ac.shape[1]):
            diff = abs(n - n_p1)
            ac[n, n_p1] = int(round(max(0, (n - 125) / 400 * n**(0.5 + diff / 50.0))))
    return ac

In [ ]:
family_size, family_choices, family_slots_matrix = input_data()
PCOSTM = get_preference_cost_matrix(pd.read_csv(family_data_path, index_col='family_id'), family_size)
ACOSTM = get_accounting_cost_matrix()

In [ ]:
class Cost:
    def __init__(self, family_data_path = family_data_path):
        family = pd.read_csv(family_data_path, index_col='family_id')
        self.family_size = family.n_people.values.astype(np.int8)
        self.family_cost_matrix = self._penality_array(family, self.family_size)
        self.accounting_cost_matrix = self._accounting_cost_matrix(family)
        
    def _penality_array(self, family, family_size):
        penalties = np.asarray([
            [
                0,
                50,
                50 + 9 * n,
                100 + 9 * n,
                200 + 9 * n,
                200 + 18 * n,
                300 + 18 * n,
                300 + 36 * n,
                400 + 36 * n,
                500 + 36 * n + 199 * n,
                500 + 36 * n + 398 * n
            ] for n in range(family_size.max() + 1)
        ])
        family_cost_matrix = np.concatenate(family.n_people.apply(
                lambda n: np.repeat(penalties[n, 10], 100).reshape(1, 100)))
        for fam in family.index:
            for choice_order, day in enumerate(family.loc[fam].drop("n_people")):
                family_cost_matrix[fam, day - 1] = penalties[family.loc[fam, "n_people"], choice_order]
        return family_cost_matrix
        
    
    def _accounting_cost_matrix(self, family):
        accounting_cost_matrix = np.zeros((500, 500))
        for n in range(accounting_cost_matrix.shape[0]):
            for diff in range(accounting_cost_matrix.shape[1]):
                accounting_cost_matrix[n, diff] = max(0, (n - 125.0) / 400.0 * n**(0.5 + diff / 50.0))
        return accounting_cost_matrix
    
    def calculate(self, prediction):
        p, ac, nl, nh = self._calculate(prediction, self.family_size, self.family_cost_matrix, self.accounting_cost_matrix)
        return (p + ac) + (nl + nh) * 1000000
        
    @staticmethod
#     @njit(fastmath=True)
    def _calculate(prediction, family_size, family_cost_matrix, accounting_cost_matrix):
        N_DAYS = 100
        MAX_OCCUPANCY = 300
        MIN_OCCUPANCY = 125
        penalty = 0
        daily_occupancy = np.zeros(N_DAYS + 1, dtype=np.int16)
        for i, (pred, n) in enumerate(zip(prediction, family_size)):
            daily_occupancy[pred - 1] += n
            penalty += family_cost_matrix[i, pred - 1]

        accounting_cost = 0
        n_low = 0
        n_high = 0
        daily_occupancy[-1] = daily_occupancy[-2]
        for day in range(N_DAYS):
            n_next = daily_occupancy[day + 1]
            n = daily_occupancy[day]
            n_high += (n > MAX_OCCUPANCY) 
            n_low += (n < MIN_OCCUPANCY)
            diff = abs(n - n_next)
            accounting_cost += accounting_cost_matrix[n, diff]

        return np.asarray([penalty, accounting_cost, n_low, n_high])

In [ ]:
class ObjectivePrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0
        self.cost = Cost()
        
    def on_solution_callback(self):
        submission = pd.read_csv(submission_path, index_col='family_id')
        for fam in all_families:
            for day in all_days:
                if solver.Value(assign[(fam, day)]):
                    submission.loc[fam, 'assigned_day'] = day
        total_cost = cost.calculate(submission['assigned_day'].values)
        print('Solution %i, time = %f s, objective = %i' %
                 self.__solution_count, self.WallTime(), total_cost)
        self.__solution_count +=1

In [ ]:
def santa_model():
    assign = {}
    n = {}
    n_bool = {}
    n_bool_next = {}

    model = cp_model.CpModel()

    for fam in all_families:
        for day in all_days:
            assign[(fam, day)] = model.NewBoolVar('x[%i][%i]' % (fam, day))

    for fam in all_families:
        model.Add(sum(assign[(fam, day)] for day in all_days 
                      if family_slots_matrix[fam, day] > 0) == 1)
        model.Add(sum(assign[(fam, day)] for day in all_days 
                      if family_slots_matrix[fam, day] == 0) == 0)

    for day in all_days:
       model.Add(sum(assign[(fam, day)] * family_size[fam] for fam in all_families
                     if family_slots_matrix[fam, day]) >= 125)
       model.Add(sum(assign[(fam, day)] * family_size[fam] for fam in all_families
                     if family_slots_matrix[fam, day]) <= 300)
    
    model.Add(sum(PCOSTM[fam, day] * assign[fam, day] for fam in all_families for day in all_days) == 62868)
#     model.Minimize(sum(PCOSTM[fam, day] * assign[fam, day] for fam in all_families for day in all_days)
#                   + sum(int(ACOSTM[day, day+1]) * n_bool[(day, i)] * n_bool_next[(day, i)] for day in all_days for i in range(1500)))
    
    
    for fam in all_families:
        model.AddDecisionStrategy([assign[(fam, family_slots_matrix[fam, 0])]], cp_model.CHOOSE_FIRST, cp_model.SELECT_MAX_VALUE)
    
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 31000
    solution_printer = ObjectivePrinter()                
    status = solver.SolveWithSolutionCallback(model, solution_printer)
    print(solver.ResponseStats())

    if status == cp_model.FEASIBLE:
        print('Feasible solution found')
    else:
        print('Not found a solution')
    print('Objective: %i' % solver.ObjectiveValue())
    print('Statistics')
    print('  - conflicts : %i' % solver.NumConflicts())
    print('  - branches  : %i' % solver.NumBranches())
    print('  - wall time : %f s' % solver.WallTime())
    return assign, solver

In [ ]:
assign, solver = santa_model()

In [ ]:
submission = pd.read_csv(submission_path, index_col='family_id')
for fam in all_families:
    for day in all_days:
        if solver.Value(assign[(fam, day)]):
            submission.loc[fam, 'assigned_day'] = day
submission.to_csv('submission.csv')

In [ ]:

# all_families = range(N_FAMILIES)
# all_days = range(N_DAYS)

# minutes_to_run = 0.2
# assign = {}
# n = {}
# n_bool = {}
# n_bool_next = {}

# model = cp_model.CpModel()

# for fam in all_families:
#     for day in all_days:
#         assign[(fam, day)] = model.NewBoolVar('x[%i][%i]' % (fam, day))

# for fam in all_families:
#     model.Add(sum(assign[(fam, day)] for day in all_days 
#                   if family_slots_matrix[fam, day] > 0) == 1)
#     model.Add(sum(assign[(fam, day)] for day in all_days 
#                   if family_slots_matrix[fam, day] == 0) == 0)

# for day in all_days:
#     n[day] = model.NewIntVar(0, 1500, 'n[%i]' % day)
#     model.Add(n[day] == sum(assign[(fam, day)] * family_size[fam] for fam in all_families 
#                   if family_slots_matrix[fam, day] > 0))
#     model.Add(n[day] >= 125)
#     model.Add(n[day] <= 300)

# n[N_DAYS] = model.NewIntVar(0, 1250, 'n[%i]' % N_DAYS)
# model.Add(n[N_DAYS] == n[N_DAYS-1])

# for day in all_days:
#     for i in range(1500):
#         n_bool[(day, i)] = model.NewBoolVar('n_bool[%i][%i]' % (day, i))
#         n_bool_next[(day, i)] = model.NewBoolVar('n_bool_next[%i][%i]' % (day, i))
#         model.Add(i == n[day]).OnlyEnforceIf(n_bool[(day, i)])
#         model.Add(i == n[day+1]).OnlyEnforceIf(n_bool_next[(day, i)])



In [ ]:
# model.Minimize(sum(int(ACOSTM[day, day+1]) * n_bool[(day, i)] * n_bool_next[(day, i)] for day in all_days for i in range(1500))
#               +sum(PCOSTM[fam, day] * assign[fam, day] for fam in all_families for day in all_days))

In [ ]:
# for day in all_days:   
#     for fam in all_families:
#         model.AddDecisionStrategy([assign[(fam, day)]], cp_model.CHOOSE_FIRST, cp_model.SELECT_MAX_VALUE)

# solver = cp_model.CpSolver()

# solver.parameters.max_time_in_seconds = minutes_to_run * 60.0
# solution_printer = ObjectivePrinter()                
# status = solver.SolveWithSolutionCallback(model, solution_printer)
# print(solver.ResponseStats())

# if status == cp_model.FEASIBLE:
#     print('Feasible solution found')
# else:
#     print('Not found a solution')
# print('Objective: %i' % solver.ObjectiveValue())
# print('Statistics')
# print('  - conflicts : %i' % solver.NumConflicts())
# print('  - branches  : %i' % solver.NumBranches())
# print('  - wall time : %f s' % solver.WallTime())

In [ ]:

# for day in all_days:
# #    model.Add(sum(assign[(fam, day)] * family_size[fam] for fam in all_families
# #                  if family_slots_matrix[fam, day]) >= 125)
# #    model.Add(sum(assign[(fam, day)] * family_size[fam] for fam in all_families
# #                  if family_slots_matrix[fam, day]) <= 300)
#     n[day] = model.NewIntVar(0, 1500, 'n[%i]' % day)
#     model.Add(n[day] == sum(assign[(fam, day)] * family_size[fam] for fam in all_families 
#                   if family_slots_matrix[fam, day] > 0))
#     model.Add(n[day] >= 125)
#     model.Add(n[day] <= 300)
# n[n_days] = model.NewIntVar(0, 1250, 'n[%i]' % n_days)
# model.Add(n[n_days] == n[n_days-1])







# acc_cost = {}
# diff = {}
# abs_diff = {}
# pow_val_base = {}
# pow_val = {}
# top = {}
# base = {}
# pow_toggle = {}

# for day in all_days:
#     top[day] = model.NewIntVar(0, 28125, 'top[%i]' % day)
#     model.Add(top[day] == (n[day] - 125) * 25)

#     diff[day] = model.NewIntVar(-1250, 1250, 'diff[%i]' % day)
#     model.Add(diff[day] == n[day]-n[day+1])
#     abs_diff[day] = model.NewIntVar(0, 1250, 'abs_diff[%i]' % day)
#     model.AddAbsEquality(abs_diff[day], diff[day])

# #     pow_val_base[day] = model.NewIntVar(1, 2501, 'pow_val_base[%i]' % day)
# #     model.Add(pow_val_base[day] == 1 + 2*abs_diff[day])
# #     pow_val[day] = model.NewIntVar(0, 3, 'pow_val[%i]' % day)
# #     model.AddDivisionEquality(pow_val[day], pow_val_base[day], 100)
# #     for i in range(4):
# #         pow_toggle[(day, i)] = model.NewBoolVar('pow_toggle[%i][%i]' % (day, i))
# #         model.Add(pow_val[day] == i).OnlyEnforceIf(pow_toggle[(day, i)])
# #         model.Add(pow_val[day] != i).OnlyEnforceIf(pow_toggle[(day, i)].Not())

#     base[day] = model.NewIntVar(1, 1250, 'base[%i]' % day) #1953125000
# #     model.Add(base[day] == 1).OnlyEnforceIf(pow_toggle[(day, 0)])
#     model.Add(base[day] == 1)#.OnlyEnforceIf(pow_toggle[(day, 1)])
# #     model.AddMultiplicationEquality(base[day], [n[day], n[day]]).OnlyEnforceIf(pow_toggle[(day, 2)])
# #     model.AddMultiplicationEquality(base[day], [n[day], n[day], n[day]]).OnlyEnforceIf(pow_toggle[(day, 3)])

#     acc_cost[day] = model.NewIntVar(0, 28126, 'acc_cost[%i]' % day)
# #     model.Add(acc_cost[day] == top[day])
#     model.AddDivisionEquality(acc_cost[day], top[day], base[day])

# model.Minimize((sum((0 * assign[(fam, family_choices[fam][0]-1)])
#         + (50 * assign[(fam, family_choices[fam][1]-1)])
#         + ((50 + 9 * family_size[fam]) * assign[(fam, family_choices[fam][2]-1)])
#         + ((100 + 9 * family_size[fam]) * assign[(fam, family_choices[fam][3]-1)])
#         + ((200 + 9 * family_size[fam]) * assign[(fam, family_choices[fam][4]-1)])
#         + ((200 + 18 * family_size[fam]) * assign[(fam, family_choices[fam][5]-1)])
#         + ((300 + 18 * family_size[fam]) * assign[(fam, family_choices[fam][6]-1)])
#                            for fam in all_families) * 10000)
#               + sum(acc_cost)
#               )
